# EDA of diabetes_012_health_indicators_BRFSS2015

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_raw = pd.read_csv("../data/split/train_raw_split.csv")

## First glance at raw data

In [ ]:
display(df_raw.shape)

In [ ]:
display(df_raw.info())

#### Observation
* There are no missing values.
* All columns contain numbers, although some should be interpreted as categories.

In [ ]:
step = 8

for i in range(0, len(df_raw.columns), step):
    display(df_raw[df_raw.columns[i : i + step]].head())

In [ ]:
display(df_raw[df_raw.select_dtypes(include="number").columns].describe())

#### Observation
* `BMI` has a very high maximum. Possibly an outlier.
* Most features are severely skewed. This is to be expected in a clinical study.

## Converted to intended datatype

### Data Dictionary – Diabetes Health Indicators

| Column                | Datatype   | Description                                                                 |
|-----------------------|------------|-----------------------------------------------------------------------------|
| `Diabetes_012`        | nominal    | 0 = no diabetes, 1 = prediabetes, 2 = diabetes                              |
| `HighBP`              | nominal    | 0 = no high blood pressure, 1 = high blood pressure                         |
| `HighChol`            | nominal    | 0 = no high cholesterol, 1 = high cholesterol                               |
| `CholCheck`           | nominal    | 0 = no cholesterol check in 5 years, 1 = yes                                |
| `BMI`                 | float      | Body Mass Index                                                             |
| `Smoker`              | nominal    | Smoked ≥100 cigarettes in lifetime: 0 = no, 1 = yes                         |
| `Stroke`              | nominal    | Ever told had a stroke: 0 = no, 1 = yes                                     |
| `HeartDiseaseorAttack`| nominal    | CHD or MI history: 0 = no, 1 = yes                                          |
| `PhysActivity`        | nominal    | Physical activity (last 30 days, non-job): 0 = no, 1 = yes                  |
| `Fruits`              | nominal    | Consumes fruit ≥1×/day: 0 = no, 1 = yes                                     |
| `Veggies`             | nominal    | Consumes vegetables ≥1×/day: 0 = no, 1 = yes                                |
| `HvyAlcoholConsump`   | nominal    | Heavy drinker: 0 = no, 1 = yes                                              |
| `AnyHealthcare`       | nominal    | Has health coverage: 0 = no, 1 = yes                                        |
| `NoDocbcCost`         | nominal    | Missed doctor due to cost (last 12 months): 0 = no, 1 = yes                 |
| `GenHlth`             | ordinal    | General health: 1 = excellent, ..., 5 = poor                                |
| `MentHlth`            | int        | Days of poor mental health (last 30 days), 0–30                             |
| `PhysHlth`            | int        | Days of poor physical health (last 30 days), 0–30                           |
| `DiffWalk`            | nominal    | Serious difficulty walking/stairs: 0 = no, 1 = yes                          |
| `Sex`                 | nominal    | 0 = female, 1 = male                                                        |
| `Age`                 | ordinal    | Age category: 1 = 18–24, ..., 13 = 80+                                      |
| `Education`           | ordinal    | Education level: 1 = none to kindergarten, ..., 6 = college grad           |
| `Income`              | ordinal    | Income level: 1 = < $10k, ..., 8 = ≥ $75k                                   |



### Conversion

In [ ]:
cat_cols = [
    "Diabetes_012",
    "HighBP",
    "HighChol",
    "CholCheck",
    "Smoker",
    "Stroke",
    "HeartDiseaseorAttack",
    "PhysActivity",
    "Fruits",
    "Veggies",
    "HvyAlcoholConsump",
    "AnyHealthcare",
    "NoDocbcCost",
    "GenHlth",
    "DiffWalk",
    "Sex",
    "Age",
    "Education",
    "Income",
]

df_raw[cat_cols] = df_raw[cat_cols].astype("int").astype("category")

## Info, Describe, Overview

In [ ]:
df_raw.info()

In [ ]:
display(df_raw.loc[:, df_raw.select_dtypes(include="number").columns].describe())

In [ ]:
def overview(df):
    """
    Creates and prints an overview of the DataFrame including data types, counts, missing values,
    unique values, and some basic statistics.
    """
    from pandas.api.types import is_numeric_dtype

    def normalized_entropy_cat(series: pd.Series) -> float:
        """
        Compute the normalized Shannon entropy of a categorical distribution.

        | Entropy in [0,1]| Interpretation         | Example class distribution   |
        | --------------- | ---------------------- | ---------------------------- |
        | 0.00 - 0.20     | Extremely imbalanced   | e.g. 99% / 1%                |
        | 0.20 - 0.40     | Strongly imbalanced    | e.g. 90% / 10% or 80/10/10   |
        | 0.40 - 0.60     | Moderately imbalanced  | e.g. 70/30 or 60/20/20       |
        | 0.60 - 0.80     | Slightly imbalanced    | e.g. 50/25/25                |
        | 0.80 - 1.00     | Balanced               | e.g. 33/33/33 or 25/25/25/25 |


        Returns 0 if only one class is present, 1 for perfectly uniform distribution.
        """
        counts = series.value_counts(normalize=True)
        entropy = -np.sum(counts * np.log2(counts))
        max_entropy = np.log2(len(counts)) if len(counts) > 1 else 1
        return entropy / max_entropy

    display(
        pd.DataFrame(
            {
                "dtype": df.dtypes,
                "total": df.count(),
                "missing": df.isna().sum(),
                "missing%": df.isna().mean() * 100,
                "n_uniques": df.nunique(),
                "uniques%": df.nunique() / df.shape[0] * 100,
                "uniques": [sorted(df[col].unique()) for col in df.columns],
                "non-numeric": [
                    list(
                        df[col][pd.to_numeric(df[col], errors="coerce").isna()].unique()
                    )
                    for col in df.columns
                ],
                "dev from mean": [
                    (
                        (
                            round(
                                ((df[col].mean() - df[col].min()) / df[col].std()), 1
                            ),
                            round(
                                ((df[col].max() - df[col].mean()) / df[col].std()), 1
                            ),
                        )
                        if is_numeric_dtype(df[col])
                        else pd.NA
                    )
                    for col in df.columns
                ],
                "most/least freq": [
                    (
                        (
                            {
                                df[col].value_counts().index[i]: list(
                                    df[col].value_counts()
                                )[i] for i in (0, -1)
                            }
                           
                        )
                        if not is_numeric_dtype(df[col])
                        else pd.NA
                    )
                    for col in df.columns
                ],
                "norm entropy": [
                    round(normalized_entropy_cat(df[col]), 2)
                    if isinstance(df[col].dtype, pd.CategoricalDtype)
                    else pd.NA
                    for col in df.columns
                ],
            }
        )
    )


overview(df_raw)

#### Observation
* `Age`, `Education`, `Income`, `Diabetes_012` are categories with more than two values. All other categories are logical.
* The target `Diabetes_012` is imbalanced.
* `BMI` contains values further than 10 standard deviations from the mean. Check for outliers.
* `CholCheck`, `Stroke`, `HvyAlcoholConsump`,  `HvyAlcoholConsump` are strongly unbalanced.
* `HeartDiseaseorAttack`, `NoDocbcCost` are moderately unbalanced.

## Crosstabs

#### Observation
* Most logical categories show a difference of more than 5 percentage points of healthy people between the two categories.
* Nominal categories suggest a correlation between increasing values (worse health, higher education and income) and the occurence of diabetes.

In [ ]:
def catplot(df, target_variable_name):
    """Display a barplot of a normalized crosstab between x and y and an absolute crosstab for a sanity check.
    ARGS:
        x: Crosstab 'index' column
        y: Crosstab 'columns' column (default: eda['Infected'])
    RETURNS: None
    """

    cat_cols = [
        c
        for c in df.select_dtypes(include="category").columns
        if c != target_variable_name
    ]

    ncols = 2
    fig, ax = plt.subplots(
        nrows=len(cat_cols), ncols=ncols, figsize=(10, len(cat_cols) * 1.5 + 2)
    )

    if len(cat_cols) == 1 and ncols == 1:
        ax = np.array([[ax]])
    elif len(cat_cols) == 1:
        ax = np.atleast_2d(ax)
    elif ncols == 1:
        ax = np.atleast_2d(ax).T

    for idx, col in enumerate(cat_cols):
        # Create two crosstabs: One for absolute and one for relative distributions
        crosstab_abs = pd.crosstab(index=df[col], columns=df[target_variable_name])
        crosstab_rel = pd.crosstab(
            index=df[col], columns=df[target_variable_name], normalize="index"
        )

        # Plot them side by side
        crosstab_abs.plot(
            kind="bar", ax=ax[idx, 0], title=f"Absolute Distribution of {col}", legend=False
        )
        crosstab_rel.plot(
            kind="bar", ax=ax[idx, 1], title=f"Relative Distribution of {col}", legend=False
        )
        ax[idx, 1].legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
        for p in ax[idx, 1].patches:
            height = p.get_height()
            if not pd.isna(height) and height > 0:
                ax[idx, 1].annotate(
                    f"{height:.2f}",
                    (p.get_x() + p.get_width() / 2, height),
                    ha="center",
                    va="bottom",
                    fontsize=8
                )

    plt.tight_layout()
    plt.show()


In [ ]:
catplot(df_raw, "Diabetes_012")

In [ ]:
%%time

categorical_features = df_raw.select_dtypes(include=["object", "category"]).columns.tolist()

df_raw_copy = df_raw[categorical_features].copy()

categorical_features = [col for col in categorical_features if col != "Diabetes_012"]
# categorical_features= ["HighBP"]
feature_ct = len(categorical_features)

fig, axs = plt.subplots(feature_ct, 1, figsize=(14, 2 * feature_ct))
if feature_ct == 1:
    axs = [axs]  # ensure axs is always iterable

for ax, feature in zip(axs, categorical_features):

    ct = pd.crosstab(index=df_raw_copy[feature], columns=df_raw_copy["Diabetes_012"])

    ct_ratio = ct.div(ct.sum(axis=1), axis=0).fillna(0)

    ct_ratio.plot(kind="barh", stacked=True, ax=ax, legend=False)

    for i, (index, row) in enumerate(ct_ratio.iterrows()):
        ratio = row.get(0, 0)
        ax.text(1.01, i, f"{ratio:.1%}", va="center", fontsize=8)

    ax.set_xlabel("ratio")
    ax.set_ylabel(feature)
    ax.set_title(f"ratio of Diabetes_012 in {feature}")
    ax.legend(title="Diabetes_012", loc="lower left")

fig.tight_layout()
plt.show()

# df_raw_copy is not needed anymore
del df_raw_copy

## Pairplots

### Observation
* No relevant differences are visible between the pairplots
* `MenHlth` and `PhysHlth` have more entries on counts divisible by 5. Possibly just the human need for beauty.

### Numeric columns

In [ ]:
%%time
sns.pairplot(data=df_raw, plot_kws={'alpha': 0.1, "s": 10}, hue='Diabetes_012')
plt.suptitle("Pairplots of all numeric features", y=1.02)
plt.show();

### Classes

In [ ]:
%%time
sns.pairplot(data=df_raw[df_raw["Diabetes_012"] != 0], plot_kws={'alpha': 0.1, "s": 10}, hue='Diabetes_012')
plt.suptitle("Pairplots of all numeric features for diabetes", y=1.02)
plt.show();

In [ ]:
%%time
sns.pairplot(data=df_raw[df_raw["Diabetes_012"] == 1], plot_kws={'alpha': 0.1, "s": 10}, hue='Diabetes_012')
plt.suptitle("Pairplots of all numeric features for diabetes", y=1.02)
plt.show();

### Sex

#### Pairplots Women

In [ ]:
%%time
sns.pairplot(data=df_raw[df_raw["Sex"] == 0], plot_kws={'alpha': 0.1, "s": 10}, hue='Diabetes_012')
plt.suptitle("Pairplots of numeric features for women", y=1.02)
plt.show();

#### Pairplots Men

In [ ]:
%%time
sns.pairplot(data=df_raw[df_raw["Sex"] == 1], plot_kws={'alpha': 0.1, "s": 10}, hue='Diabetes_012', )
plt.suptitle("Pairplots of numeric features for men", y=1.02)
plt.show();

## Correlations

#### Observations
* There are notable correlations between 
  * `GenHlth`, `PhysHlth` and `DiffWalk`. These seem reasonable.
  * `Education` and `Income`. Also reasonable.
* There are lower, but still notable correlations between
  * `Diabetes_012`, `GenHlth`
  * `GenHlth`, `Diabetes_012`, `HighBP`, `Income`
  * `MenHlth`, `GenHlth`
  * `MenHlth`, `PhysHlth`
  * `Income`, `GenHlth`, `DiffWalk`, last one somewhat surprising

#### Conclusion
Relevant correlations are present. Appropriate feature selection must be conducted.

In [ ]:
corr = df_raw.corr()
mask = np.abs(corr) < 0.3
annot = corr.round(2).astype(str)
annot[mask] = ""

plt.figure(figsize=(16, 13))
sns.heatmap(
    corr,
    annot=annot,
    fmt="",
    cmap="coolwarm",
    vmin=-1,
    vmax=1,
)

plt.title("Correlations between all columns", y=1.02)
plt.show();

## Outlier Detection

#### Observations
* Up to about 70 `BMI` has a clean distribution. From then on spikes appear with no visible reason. The distribution of diabetes looks similar to the whole distribution.
* `MentHlth` and `PhysHlth` have most of their values at 0. The distribution of diabetes looks similar to the whole distribution.

#### Conclusion
* Check the validity of the very high BMI-values.
* No outliers found.

In [ ]:
from statsmodels.robust import mad
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def mark_outliers_mad(
    df,
    std=3,
    show_cum=False,
    show_interesting_rows=False,
    interesting_rows=None,
    return_masks=False
):
    """
    Detect and visualize outliers in numeric columns using the MAD (Median Absolute Deviation) method.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing the data.
    std : float, optional
        Threshold in MAD units to define outliers (default is 3).
    show_cum : bool, optional
        If True, show cumulative distribution plots with outlier thresholds.
    show_interesting_rows : bool, optional
        If True, show histogram restricted to interesting_rows.
    interesting_rows : pd.Series[bool], optional
        Boolean mask indicating rows of interest.
    return_masks : bool, optional
        If True, return dictionary of boolean masks marking outliers per column.

    Returns
    -------
    dict (optional)
        Dictionary of boolean Series with outlier masks for each numeric column.
    """
    def get_mad_outliers_mask(df, column, std=3):
        x = df.loc[df[column].notna(), column]
        outliers = (abs(x - x.median()) / mad(x)) >= std

        mask = pd.Series(False, index=df.index)
        mask.loc[x.index] = outliers
        return mask

    outlier_masks = {}
    numeric_cols = df.select_dtypes(include="number").columns.tolist()

    column_config = {col: {} for col in numeric_cols}

    show_interesting_rows = show_interesting_rows and (interesting_rows is not None)

    ncols = 1 + sum([show_cum, show_interesting_rows])
    fig, ax = plt.subplots(nrows=len(numeric_cols), ncols=ncols, figsize=(14, len(numeric_cols)*2 + 2))

    if len(numeric_cols) == 1 and ncols == 1:
        ax = np.array([[ax]])
    elif len(numeric_cols) == 1:
        ax = np.atleast_2d(ax)
    elif ncols == 1:
        ax = np.atleast_2d(ax).T

    for idx, col in enumerate(numeric_cols):
        config = column_config[col]
        axe = ax[idx, 0]
        outlier_mask = get_mad_outliers_mask(df, col, std)

        if return_masks:
            outlier_masks[col] = outlier_mask

        df_temp = pd.concat(
            [df, pd.DataFrame({f"mad_{std}_outlier": outlier_mask})], axis=1)

        sns.histplot(data=df_temp, x=col, ax=axe, hue=f"mad_{std}_outlier")
        axe.set_yscale("log")
        axe.set_title(f"Histogram: {col}")

        if show_interesting_rows:
            prev_axe = axe
            axe = ax[idx, 1]
            axe.set_xlim(prev_axe.get_xlim())
            axe.set_ylim(prev_axe.get_ylim())

            df_temp = pd.concat(
                [df[[col]].copy(), pd.DataFrame({f"mad_{std}_outlier": outlier_mask})], axis=1)
            df_temp.loc[~interesting_rows, col] = np.nan

            sns.histplot(data=df_temp, x=col, ax=axe, hue=f"mad_{std}_outlier")
            axe.set_yscale("log")
            axe.set_title(f"Subset histogram: {col}")

        if show_cum:
            lower_threshold = df[~outlier_mask][col].min()
            upper_threshold = df[~outlier_mask][col].max()

            axe = ax[idx, 1 + show_interesting_rows]
            sns.ecdfplot(df[col].dropna(), ax=axe)

            axe.axvline(x=lower_threshold, color="blue", linestyle="--", label="lower threshold")
            axe.axvline(x=upper_threshold, color="red", linestyle="--", label="upper threshold")

            col_data = df[col].dropna()
            outlier_low = (col_data < lower_threshold).mean()
            outlier_high = (col_data > upper_threshold).mean()

            axe.set_title(
                f"CDF: {col}\n"
                f"Outliers < {lower_threshold:.2f}: {outlier_low:.3f} | "
                f"> {upper_threshold:.2f}: {outlier_high:.3f}"
            )
            axe.legend()


    plt.suptitle(f"MAD Outlier Detection (±{std} MAD)", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

    if return_masks:
        return outlier_masks


In [ ]:
mark_outliers_mad(df_raw, show_interesting_rows=True, interesting_rows=df_raw['Diabetes_012']!=0, show_cum=True)